In [1]:
# pip install -U langchain langchain-openai langchain-community faiss-cpu pypdf python-dotenv langsmith

import os
from dotenv import load_dotenv

from langsmith import traceable  # <-- key import

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [2]:

# --- LangSmith env (make sure these are set) ---
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_API_KEY=...
# LANGCHAIN_PROJECT=pdf_rag_demo

load_dotenv()

os.environ['LANGCHAIN_PROJECT'] = "RAG Chatbot"


In [3]:
PDF_PATH = "resume.pdf"

In [4]:
# ---------- traced setup steps ----------
@traceable(name="load_pdf")
def load_pdf(path: str):
    loader = PyPDFLoader(path)
    return loader.load()  # list[Document]


In [5]:
@traceable(name="split_documents")
def split_documents(docs, chunk_size=1000, chunk_overlap=150):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)


In [6]:
@traceable(name="build_vectorstore")
def build_vectorstore(splits):
    emb = OpenAIEmbeddings(model="text-embedding-3-small", base_url="https://openrouter.ai/api/v1")
    # FAISS.from_documents internally calls the embedding model:
    vs = FAISS.from_documents(splits, emb)
    return vs

In [7]:
# You can also trace a “setup” umbrella span if you want:
@traceable(name="setup_pipeline")
def setup_pipeline(pdf_path: str):
    docs = load_pdf(pdf_path)
    splits = split_documents(docs, chunk_size=100, chunk_overlap=50)
    vs = build_vectorstore(splits)
    return vs


In [8]:
# ---------- pipeline ----------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    base_url="https://openrouter.ai/api/v1"
)


In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer ONLY from the provided context. If not found, say you don't know."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

In [10]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)


In [11]:
# Build the index under traced setup
vectorstore = setup_pipeline(PDF_PATH)

In [12]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})


In [13]:
parallel = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})

In [14]:
chain = parallel | prompt | llm | StrOutputParser()

In [15]:

# ---------- run a query (also traced) ----------
print("PDF RAG ready. Ask a question (or Ctrl+C to exit).")
q = input("\nQ: ").strip()

PDF RAG ready. Ask a question (or Ctrl+C to exit).



Q:  do i ever work in dd


In [16]:
# Give the visible run name + tags/metadata so it’s easy to find:
config = {
    "run_name": "pdf_rag_query"
}

In [17]:
ans = chain.invoke(q, config=config)
print("\nA:", ans)


A: I don't know.
